In [ ]:
#Initial utility methods and model parameters

import numpy as np
import pandas as pd
from enum import IntEnum


#Utility methods

#clip01 keeps all continuous variables in the simulation bounded between [0,1]
def clip01(x):
  return np.clip(x, 0.0, 1.0)

# Enums for ensuring clean and explicit model events
class Action(IntEnum):
  PASSIVE = 0 # NIST CSF 2.0 Functions Identify + Protect
  ACTIVE = 1 # NIST CSF 2.0 Functions Detect + Respond
  RECOVER = 2 #Recover function

class AttackTarget(IntEnum):
  NONE = 0
  IT = 1
  OT = 2

class Intensity(IntEnum):
  LOW = 0
  HIGH = 1

# Model Parameters
Parameters = pd.Series({
    #Simulation Control
    "T" : 500,
    'Seed': 1,

    #Governance
    "G" : 0.6,

    #Initial Defender State variables
    "it_vuln_init" : 0.6,
    "ot_vuln_init" : 0.7,
    "id_cap_init" : 0.2,
    'it_comp_init' : 0,
    'ot_comp_init' : 0,
    'downtime_init' : 0.0,
    'phys_damage_init' : 0.0,

    #Attacker event process
    'p_attack': 0.35,
    'p_ot_given_attack_base' : 0.35,
    'p_ot_bonus_if_it_comp' : 0.20,
    'p_ot_bonus_if_ot_high_vuln' : 0.20,
    'ot_high_vuln_threshold': 0.7,

    #Attack intensity deterrence via identification
    'p_high_base' : 0.50,
    'k_deterrence' : 2.0,
})

#Reproducible randomness
rng = np.random.default_rng(int(Parameters["Seed"]))

In [ ]:
#Simulation State Code

#Initialize state as a Pandas series
State = pd.Series({
    'it_vuln' : clip01(Parameters['it_vuln_init']),
    'ot_vuln' : clip01(Parameters['ot_vuln_init']),
    'id_cap' : clip01(Parameters['id_cap_init']),

    #defender compromised treated as boolean, flags [0,1] indicate whether defender IT or OT is compromised
    "it_comp" : int(Parameters['it_comp_init']),
    "ot_comp" : int(Parameters['ot_comp_init']),
    "downtime" : float(Parameters['downtime_init']),
    "phys_damage" : float(Parameters['phys_damage_init']),
})

#Simulation time + log storage data structures
t = 0
rows = []

def gov_mult(Parameters):
  #baseline government multiplier = 0.5 + 0.5 * G
  return 0.5 + 0.5 * clip01(Parameters['G'])


def snapshot_state(Parameters, State, t):
  #returns a dictionary of the pre-action state we wish to record in the log
  return{
      't' : t,
      'G' : float(clip01(Parameters['G'])),
      'govt_mult': float(gov_mult(Parameters)),

      'it_vuln' : float(State['it_vuln']),
      'ot_vuln' : float(State['ot_vuln']),
      'id_cap' : float(State['id_cap']),

      'it_comp' : int(State['it_comp']),
      'ot_comp' : int(State['ot_comp']),
      'downtime' : float(State['downtime']),
      'phys_damage' : float(State['phys_damage']),
  }

def sim_step(Parameters, State, rng, t, rows):
  pre = snapshot_state(Parameters, State, t)

  #Placeholder policy: always PASSIVE for now (1/6/26)
  action = Action.PASSIVE

  #Log row (attacker/outcomes will be added later)
  row = dict(pre)
  row.update({
     'action' : int(action),
    'action_name' : action.name,
  })
  rows.append(row)

  return t + 1 #advance time


def run_sim(Parameters, State, rng):
  rows_local = []
  t_local = 0

  for _ in range(int(Parameters['T'])):
    t_local = sim_step(Parameters, State, rng, t_local, rows_local)

  return pd.DataFrame(rows_local)


#Debugging test to make sure Dataframe with T rows is produced
df_test = run_sim(Parameters, State.copy(), rng)
df_test.head(10)





,t,G,govt_mult,it_vuln,ot_vuln,id_cap,it_comp,ot_comp,downtime,phys_damage,action,action_name
0,0,0.6,0.8,0.6,0.7,0.2,0,0,0.0,0.0,0,PASSIVE
1,1,0.6,0.8,0.6,0.7,0.2,0,0,0.0,0.0,0,PASSIVE
2,2,0.6,0.8,0.6,0.7,0.2,0,0,0.0,0.0,0,PASSIVE
3,3,0.6,0.8,0.6,0.7,0.2,0,0,0.0,0.0,0,PASSIVE
4,4,0.6,0.8,0.6,0.7,0.2,0,0,0.0,0.0,0,PASSIVE
5,5,0.6,0.8,0.6,0.7,0.2,0,0,0.0,0.0,0,PASSIVE
6,6,0.6,0.8,0.6,0.7,0.2,0,0,0.0,0.0,0,PASSIVE
7,7,0.6,0.8,0.6,0.7,0.2,0,0,0.0,0.0,0,PASSIVE
8,8,0.6,0.8,0.6,0.7,0.2,0,0,0.0,0.0,0,PASSIVE
9,9,0.6,0.8,0.6,0.7,0.2,0,0,0.0,0.0,0,PASSIVE
